In [1]:
import pandas as pd
import numpy as np

import swifter #catalyst
import geoip2.database

import warnings
warnings.simplefilter(action="ignore")

## Load given dataset

In [2]:
data_article = pd.read_csv("converted_articles.csv")
data_user = pd.read_csv('Fingerprint.csv')

In [3]:
data_article.sample(5)

,total_convert,post_title,link,parent_categories,child_categories
103,1,Apa Itu <em>Social Selling</em> dan Peranannya...,https://id.techinasia.com/peran-social-selling...,Profesional,Digital Marketing
28,3,"Pendekatan Vidio yang Berbeda dari HOOQ, iFlix...",https://id.techinasia.com/pendekatan-konten-lo...,Startup,Industry Insights
198,1,Cara Finfleet Jadi Perpanjangan Tangan Layanan...,https://id.techinasia.com/finfleet-solusi-laya...,Startup,Startup Terbaru
52,2,Cara Mengelola dan Memfasilitasi Pekerja <em>R...,https://id.techinasia.com/cara-mengelola-dan-m...,Profesional,Team Management
72,2,[Update] 11 <em>Provider</em> Internet Kabel d...,https://id.techinasia.com/inilah-4-provider-in...,Startup,NaN


In [4]:
data_user.sample(5)

,id,tracking_id,name,user_id,action,post_id,post_slug,comment_id,comment_content,query,referrer,ip,device,timestamp,load_at,fingerprint
1293142,b1461443-b4f4-4f73-8d22-a1b61d35b601,5ea9205ebc186985698d6277,detail_article,0,read,188210.0,kasus-peretasan-populer-indonesia,NaN,NaN,NaN,https://www.google.com/,182.1.125.27,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2020-04-29 06:36:14.18,2020-04-30 04:00:56.681966+00,701b5b0a0c4b6de2aa805ede8f509a5d
447175,839d44a5-3ca5-4763-b25b-25b546735f65,5e819a73bc18698569543e37,detail_article,0,read,251245.0,tip-dan-trik-slack,NaN,NaN,NaN,https://www.google.com/,114.124.176.255,Mozilla/5.0 (Linux; Android 9; SM-J530Y) Apple...,2020-03-30 00:12:53.579,2020-03-30 08:04:18.282714+00,7ad48605b4650208bed640f19d288825
1667490,2cf1ce62-9a79-413b-a811-5cea2acb682f,5ece8c45bc18698569b7c10d,detail_article,0,read,265364.0,gudangada-di-pasar-fmcg,NaN,NaN,NaN,https://www.google.com/,139.194.51.114,Mozilla/5.0 (Linux; Android 7.1.1; BBB100-7) A...,2020-05-27 15:50:29.671,2020-05-27 16:31:28.477053+00,3ca570e59fa7c62b243818425e04e010
464030,22b9cde3-de30-4c68-8333-893cbf16d288,5e81e1a5bc18698569554081,detail_article,0,read,153635.0,cinemoji-mari-tebak-judul-film-dengan-emoji,NaN,NaN,NaN,https://www.google.com/,36.73.35.164,Mozilla/5.0 (Linux; Android 9; Redmi Note 7) A...,2020-03-30 12:10:13.33,2020-03-30 12:32:02.567771+00,2dbfb19f1e4129c1ac70ac02033a0b2c
379102,a6c29467-6cc2-46fb-9995-d638aac2466b,5e7dc616bc186985694f7df4,detail_article,0,read,235190.0,kumpulan-plaftorm-desain-cv-terbaik,NaN,NaN,NaN,https://www.google.com/,120.188.35.23,Mozilla/5.0 (Linux; Android 8.1.0; CPH1909 Bui...,2020-03-27 09:23:34.988,2020-03-27 10:01:27.112464+00,125f16d90cd45d930ad89ae7933baacb


In [5]:
def null_uniq(dataset):
    nul_uni = []
    for i in dataset.columns:
        nul_uni.append([i, round(dataset[i].isnull().sum()/len(dataset), 3), 
                        round(dataset[i].nunique()/len(dataset), 3), dataset[i].dtype])
    return pd.DataFrame(nul_uni, columns=["column", "null", "uniqueness", "dtype"])

In [6]:
#percetage of null values each columns
#uniqueness equal to one meaning that each row has unique values
null_uniq(data_user)

,column,null,uniqueness,dtype
0,id,0.000,1.000,object
1,tracking_id,0.000,1.000,object
2,name,0.000,0.000,object
3,user_id,0.000,0.004,int64
4,action,0.000,0.000,object
5,post_id,0.094,0.008,float64
6,post_slug,0.000,0.008,object
7,comment_id,1.000,0.000,float64
8,comment_content,1.000,0.000,float64
9,query,1.000,0.000,object


## Assessing Data

In [7]:
#data size 
data_article.shape, data_user.shape

((242, 5), (2586099, 16))

In [8]:
#datetime period
data_user.timestamp.min(), data_user.timestamp.max()

('2020-03-09 06:25:20.218', '2020-08-10 06:07:31.9')

In [9]:
uniq_1 = data_article.link.str.split("/", expand=True)[3].unique()
uniq_2 = data_user.post_slug.unique()

x = []
for i, val in enumerate(uniq_1):
    if val not in uniq_2:
        x.append(i)

# the list (converter) arcticle don't in free-users dataset    
data_article.iloc[x]

#quite odd, the converter article have not read (or can't be read anymore) in given five month period

,total_convert,post_title,link,parent_categories,child_categories
45,2,Data Akuisisi Perusahaan Teknologi di Asia Ten...,https://id.techinasia.com/data-akuisisi-perusa...,Data,NaN
75,2,[Infografik] Bisnis Virtual Hotel Operator yan...,https://id.techinasia.com/infografik-bisnis-vi...,Visual Story,NaN
79,2,Apa yang Terjadi di Industri Pembayaran Indone...,https://id.techinasia.com/apa-yang-terjadi-di-...,Startup,Industry Insights
136,1,"TikTok Hapus 380.000 Video di AS, Pendapatan A...",https://id.techinasia.com/tiktok-hapus-380-000...,Brief,NaN
175,1,Cerita 3 <em>Founder</em> yang Sempat Gagal Me...,https://id.techinasia.com/cerita-kegagalan-fou...,Startup,Entrepreneur
219,1,Kisah <em>Startup</em> Lokal yang Berkembang m...,https://id.techinasia.com/kisah-sukses-startup...,Startup,Featured Startup


In [10]:
# '0' doesn't make sense
data_user["user_id"].value_counts()[:10]

0         2517584
78152        2502
140216        933
72902         759
84027         723
136           674
52010         468
9544          448
75846         445
59108         406
Name: user_id, dtype: int64

In [11]:
# only have one value
data_user["name"].value_counts(dropna=False)

detail_article    2586099
Name: name, dtype: int64

In [12]:
#the value largely dominant by read action
data_user["action"].value_counts(dropna=False)

read        2586031
klik             50
search            8
Google            4
Twitter           3
Facebook          2
comment           1
Name: action, dtype: int64

In [13]:
# too many variance which can be aggregating to smaller group
data_user["referrer"].value_counts(dropna=False).sample(5)

https://id.techinasia.com/diferensiasi-qasir-persaingan-pos                                                                                                                               5
https://www.google.co.id/search?q=ps+vita+lebih+mahal+dari+ps3&client=ucweb-lite-b&channel=sb                                                                                             1
https://id.techinasia.com/talk/final-fantasy-vi-mestinya-juga-dibuat-ulang                                                                                                                4
https://www.google.co.id/search?safe=strict&client=ucweb-b-bookmark&oq=Cara+menghubunhkan+ps+vita+ke+smart+tv&aqs=mobile-gws-lite..&source=hp&q=Cara+menghubunhkan+ps+vita+ke+smart+tv    1
https://www.google.com/url?q=https://id.techinasia.com/sikatabis-website-pembanding-kredit-pemilikan-rumah&sa=D&ust=1587930559776000&usg=AFQjCNG2Gn5vIPBzRrP8Fem11Vo9t4YUSw               1
Name: referrer, dtype: int64

In [14]:
#character such as [update], </em> should be drop
data_article.post_title.value_counts().head()

Indogen Capital Galang Dana Rp699 Miliar untuk Modal Investasi Baru di <em>Startup</em>    1
3 Tip Menyusun Strategi Pemasaran Efektif dengan Bujet Terbatas                            1
Pentingnya <em>Sales Pipeline</em> bagi <em>Startup</em> agar Meraih Target Penjualan      1
8 Tip untuk para <em>Founder</em> ketika Pertama Kali Berhadapan dengan Investor           1
Metode Tokopedia dalam Mengamankan Data 90 Juta Pelanggannya                               1
Name: post_title, dtype: int64

In [15]:
# it's ok one post id one post slug, vice versa
#thus the article_id can be one of them
data_user[data_user["post_id"] == data_user["post_id"].sample(1).values[0]]["post_slug"].value_counts()
data_user[data_user["post_slug"] == data_user["post_slug"].sample(1).values[0]]["post_id"].value_counts()

10873.0    7709
31478.0       1
Name: post_id, dtype: int64

## Cleaning process
---

### Drop useless columns and rows

In [16]:
#drop other than read
data_user = data_user[data_user["action"] == "read"].copy()

data_user.drop(columns=["id", "tracking_id", "name", "action", "post_id", "comment_id",
                        "comment_content", "query", "load_at", "fingerprint"], inplace=True)

### [data_article] Drop unnecessary char

In [17]:
# in data_article
title = data_article["post_title"].copy().str.replace(r"<[em/]+>", "").str.split("]", expand=True).applymap(lambda x: np.nan if x == None else x)
title[1] = title[1].fillna(title[0])
data_article["post_title"] = title[1]
title.sample(5)

,0,1
14,[Update,Cara Menyusun Proyeksi Keuangan Saat Startup ...
200,"Dorong Perkembangan GO-TIX, GO-JEK Akuisisi LOKET","Dorong Perkembangan GO-TIX, GO-JEK Akuisisi LOKET"
147,[Update,Kumpulan Portal Online Khusus Wanita di Indon...
150,8 Tip untuk para Founder ketika Pertama Kali B...,8 Tip untuk para Founder ketika Pertama Kali B...
179,Karakteristik Convertible Note sebagai Salah S...,Karakteristik Convertible Note sebagai Salah S...


In [18]:
#make post_slug based on link
data_article["post_slug"] = data_article.link.str.split("/", expand=True)[3]

#make link based on post_slug
data_user["link"] = "https://id.techinasia.com/" + data_user["post_slug"]

### [data_user] Correcting dtype

In [19]:
data_user.timestamp = pd.to_datetime(data_user.timestamp)
#data_user.user_id = data_user.user_id.astype("object").str.strip()

## Little bit Feature Engineering

### [data_user] Convert ip columns to location

In [20]:
#downloaded GeoLite2 database
reader = geoip2.database.Reader('GeoLite2-City_20200915/GeoLite2-City.mmdb')

def geolocation(data):
    try:
        response = reader.city(data)
        a = response.country.name
        b = response.subdivisions.most_specific.name
        c = response.city.name
        d = response.location.latitude
        e = response.location.longitude
        return [a, b, c, d, e]
    except:
        return np.nan

In [21]:
ip = pd.DataFrame(data_user["ip"].unique())
ip["location"] = ip[0].swifter.apply(geolocation)

ip["country"] = ip["location"].str[0]
ip["region"] = ip["location"].str[1]
ip["city"] = ip["location"].str[2]
ip["latitude"] = ip["location"].str[3]
ip["longitude"] = ip["location"].str[4]

ip.drop(columns="location", inplace=True)
ip.rename(columns={0:"ip"}, inplace=True)

In [22]:
# add location column to data_user dataset
data_user = pd.merge(data_user, ip, left_on="ip", right_on="ip", how="left")

### [data_user] Aggregating referrer and device columns to smaller categories

In [23]:
z = data_user["referrer"].str.split(".", expand=True)

In [24]:
# I only cleaning ten patterns
#replace com/ value with should be
com = z[z[1] == "com/"][0].str.split("/", expand=True)[2]
z[1].iloc[com.index] = com

#search not category
s = z[z[1] == "search"]
s[2] = s[2].str.replace("tb", "tb.ask")
z[1].iloc[s.index] = s[2]

#googleapis include to google category
z[1] = z[1].str.replace("googleapis", "google")
#google ads also
gog = z[z[1] == "g"][0].str.split("/", expand=True)[2].str.replace("googleads", "google_ads")
z[1].iloc[gog.index] = gog
#googleweblight also
z[1] = z[1].str.replace("googleweblight", "google_weblight")

#the category contains id is not a category
idd = z[z[1].str.contains("id", na=False)][0].str.split("/", expand=True)[2]
z[1].iloc[idd.index] = idd

#the in/ atcully linkedin category
lin = z[z[1] == "in/"][0].str.split("/", expand=True)[2].str.replace("lnkd", "linkedin")
z[1].iloc[lin.index] = lin

#convert \t
t = z[z[1].str.contains("co/", na=False)][0].str.split("/", expand=True)[2].swifter.apply(lambda x: "twitter" if x == "t" else x)
z[1].iloc[t.index] = t

#convert \co to
tw = z[z[1] == "co"][0].str.split("/", expand=True)[2].swifter.apply(lambda x: "twitter" if x == "t" else x)
z[1].iloc[tw.index] = tw

#m to wikipedia
wi = z[z[1] == "m"][2]
z[1].iloc[wi.index] = wi

#techinasia add
tech = z[z[1].str.contains("techinasia/", na=False)][0].swifter.apply(lambda x: "techinasia" if x != np.nan else x)
z[1].iloc[tech.index] = tech

#com/ to another category
com1 = z[z[1].str.contains("com/", na=False)][0].str.split("/", expand=True)[2]
z[1].iloc[com1.index] = com1

In [25]:
data_user["referrer"] = z[1].copy()

###########

In [26]:
#device columns
dev = data_user["device"].str.split(";", expand=True)

In [27]:
asd = dev[1].str.strip().swifter.apply(lambda x: x if x != "U" else "Android").str.replace(r"\s?[0-9]\s?", "").str.replace("CPU", "").str.replace("Intel", "").str.extract(r"(\w+)")[0].str.strip()

## too much variance of pattern, exclude them
data_user["device_type"] = asd.swifter.apply(lambda x: x if x in ["Android", "Win", "Mac",
                                                                  "iPhone", "Linux", "YandexBot"] else np.nan)

### [data_user] Make new user_id for "0" values

In [28]:
data_user1 = data_user[data_user["user_id"] != 0]
a = pd.DataFrame(data_user1.user_id.unique())
a["new_user_id"] = range(len(a))

In [29]:
data_user1 = pd.merge(data_user1, a, left_on="user_id", right_on=0).copy().drop(columns=[0, "user_id"])

In [30]:
data_user2 = data_user[data_user["user_id"] == 0]
b = pd.DataFrame(data_user2["ip"].unique())
b["new_user_id"] = range(len(a), len(a) + len(b))

In [31]:
data_user2 = pd.merge(data_user2, b, left_on="ip", right_on=0).copy().drop(columns=[0, "user_id"])

In [32]:
data_user = pd.concat([data_user2, data_user1])

In [33]:
data_user.to_csv("techinasia_data_user.csv")
data_article.to_csv("techinasia_data_article.csv")

----